In [1]:
"""Møller-Plesset perturbation theory to second order"""

__author__    = "Roberto Di Remigio"
__credit__    = ["Roberto Di Remigio", "Xin Li"]

__copyright__ = "(c) 2021, ENCSS and PDC"
__license__   = "MIT"
__date__      = "2021-04-14"

<figure>
  <IMG SRC="../img/ENCCS-PDC-logos.jpg" WIDTH=150 ALIGN="right">
</figure>

# Møller-Plesset perturbation theory to second order

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

## Theory refresher

Møller-Plesset perturbation theory to second order

In terms of spin-orbitals:

$$
E_{\mathrm{MP2}} =  
\frac{1}{4} 
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij \| ab \rangle \langle ab \| ij \rangle}{\varepsilon_{ij}^{ab}}
$$

where the *orbital energy denominator* is: $\varepsilon_{ij}^{ab} = \varepsilon_{i} + \varepsilon_{j} -\varepsilon_{a} - \varepsilon_{b} $.

For a closed-shell, restricted reference using real MOs:

$$
E_{\mathrm{MP2}} =
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij | ab \rangle}{\varepsilon_{ij}^{ab}}
[ 2 \langle ij | ab \rangle - \langle ij | ba \rangle ],
$$

which we can further rearrange into to two terms, *opposite-spin* and *same-spin*:

$$
E_{\mathrm{MP2}} = 
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij | ab \rangle\langle ij | ab \rangle}{\varepsilon_{ij}^{ab}} + 
\sum_{ij}^{N_{\mathrm{O}}} \sum_{ab}^{N_{\mathrm{V}}} 
\frac{\langle ij | ab \rangle[ \langle ij | ab \rangle - \langle ij | ba \rangle ]}{\varepsilon_{ij}^{ab}} = 
E_{\mathrm{MP2}}^{\mathrm{OS}} + E_{\mathrm{MP2}}^{\mathrm{SS}}.
$$

Our implementation will:

1. Obtain the reference closed-shell determinant from a Hartree-Fock calculation.
2. Transform the AO basis ERI tensor.
3. Assemble the MP2 energy correction.

We start with the declaration of the usual water molecule and its basis set. We also perform the SCF calculation with the `ScfRestrictedDriver`.

In [2]:
import veloxchem as vlx

h2o_xyz = """3
water                                                                                                                          
O    0.000000000000        0.000000000000        0.000000000000                         
H    0.000000000000        0.740848095288        0.582094932012                         
H    0.000000000000       -0.740848095288        0.582094932012
"""

mol = vlx.Molecule.from_xyz(h2o_xyz)

basis = vlx.MolecularBasis.read(mol, "6-31g")

scfdrv = vlx.ScfRestrictedDriver()
scfdrv.compute(mol, basis)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 12.
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Erro

We can now access orbital energies and MO coefficients from the driver:

In [3]:
epsilon = scfdrv.scf_tensors["E"]
C = scfdrv.scf_tensors["C"]

## The Integral transformation

We compute the MP2 energy correction with the ERI expressed in MO basis: we need to transform the ERI tensor from AO basis.
The transformation reads:

$$
\langle pq | rs \rangle = \sum_{\mu\nu\kappa\lambda} C_{\mu p}C_{\nu r} (\mu\nu|\kappa\lambda) C_{\kappa q} C_{\lambda s},
$$

with the MO integrals in [**physicists' notation**](http://vergil.chemistry.gatech.edu/notes/permsymm/permsymm.html). The transformation requires $O(N^{8})$ operation count. 
However, we can perform it more efficiently as a stepwise contraction:

$$
\langle pq | rs \rangle = \sum_{\mu} C_{\mu p}  \left(\sum_{\nu} C_{\nu r}  \left (\sum_{\kappa} \left(\sum_{\lambda} (\mu\nu|\kappa\lambda) C_{\lambda s} \right) C_{\kappa q} \right)\right).
$$

We should also note that we do **not** need the full ERI tensor in MO basis, but rather the *OOVV* class of integrals, which involve two occupied and two virtual MO indices:

$$
\langle ij | ab \rangle = 
\sum_{\mu} C_{\mu i}  
\left(\sum_{\nu} C_{\nu j}  
\left(\sum_{\kappa} 
\left(\sum_{\lambda} (\mu\kappa|\nu\lambda) C_{\lambda b} \right)
C_{\kappa a}\right)\right).
$$

In [4]:
eridrv = vlx.ElectronRepulsionIntegralsDriver()
mknl = eridrv.compute_in_mem(mol, basis)

In [5]:
import numpy as np

N_O = mol.number_of_electrons() // 2
N_V = scfdrv.mol_orbs.number_mos() - N_O

mknb = np.einsum("mknl,lB->mknB", mknl, C[:, N_O:])
print(f"{mknb.shape=}")
mnab = np.einsum("mknB,kA->mnAB", mknb, C[:, N_O:])
print(f"{mnab.shape=}")
mjab = np.einsum("mnAB,nJ->mJAB", mnab, C[:, :N_O])
print(f"{mjab.shape=}")
ijab = np.einsum("mJAB,mI->IJAB", mjab, C[:, :N_O])
print(f"{ijab.shape=}")

mknb.shape=(13, 13, 13, 8)
mnab.shape=(13, 13, 8, 8)
mjab.shape=(13, 5, 8, 8)
ijab.shape=(5, 5, 8, 8)


Let's compare our *OOVV* ERI tensor with the one computed by using VeloxChem's own `MOIntegralsDriver`:

In [6]:
moeridrv = vlx.MOIntegralsDriver()
moeri = moeridrv.compute_in_mem(mol, basis, mol_orbs=scfdrv.mol_orbs, mints_type="OOVV")

np.testing.assert_allclose(ijab, moeri, atol=1.e-10)

In [7]:
e_mp2_ss = 0.0
e_mp2_os = 0.0

# extract the occupied subset of the orbital energies
e_ij = epsilon[:N_O]
# extract the virtual subset of the orbital energies
e_ab = epsilon[N_O:]

for i in range(N_O):
    for j in range(N_O):
        for a in range(N_V):
            for b in range(N_V):
                # enegy denominators
                e_ijab =  e_ij[i] + e_ij[j] - e_ab[a] - e_ab[b]
                
                # opposite spin
                e_mp2_os += (ijab[i, j, a, b] * ijab[i, j, a, b]) / e_ijab
                
                # same spin
                e_mp2_ss += ijab[i, j, a, b] * (ijab[i, j, a, b]  - ijab[i, j, b, a]) / e_ijab 

In [8]:
print(f"Opposite-spin MP2 energy: {e_mp2_os:20.12f}")
print(f"Same-spin MP2 energy:     {e_mp2_ss:20.12f}")
print(f"MP2 energy:               {e_mp2_os + e_mp2_ss:20.12f}")

Opposite-spin MP2 energy:      -0.097649768335
Same-spin MP2 energy:          -0.029820898249
MP2 energy:                    -0.127470666584


VeloxChem has its own implementation of the MP2 energy correction. We can check our result against it.

In [9]:
mp2drv = vlx.Mp2Driver()
mp2drv.compute_conventional(mol, basis, scfdrv.mol_orbs)

np.testing.assert_allclose(e_mp2_os + e_mp2_ss, mp2drv.e_mp2, atol=1e-9)

## Bonus: using `np.einsum`

We can compute the MP2 energy terms using `np.einsum`. First, we need to define the energy denominators as a 4-index tensor with `np.reshape`.

In [10]:
e_ijab = 1 / (e_ij.reshape(N_O, 1, 1, 1) - e_ab.reshape(1, 1, N_V, 1) + e_ij.reshape(1, N_O, 1, 1) - e_ab.reshape(1, 1, 1, N_V))

The same-spin and opposite-spin are the contraction of the ERI and denominators tensors. The results are unchanged with respect to the quadruple-loop approach:

In [11]:
mp2_os = np.einsum("ijab,ijab,ijab->", ijab, ijab, e_ijab)
mp2_ss = np.einsum("ijab,ijab,ijab->", ijab, ijab - ijab.swapaxes(2, 3), e_ijab)

In [12]:
print(f"Opposite-spin MP2 energy: {mp2_os:20.12f}")
print(f"Same-spin MP2 energy:     {mp2_ss:20.12f}")
print(f"MP2 energy:               {mp2_os + mp2_ss:20.12f}")

Opposite-spin MP2 energy:      -0.097649768335
Same-spin MP2 energy:          -0.029820898249
MP2 energy:                    -0.127470666584
